In [1]:
import numpy as np
import pandas as pd

In [2]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

Using TensorFlow backend.
2021-11-12 02:51:50.897558: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [3]:
# 顯示所有列
pd.set_option('display.max_columns', None)

In [ ]:
# 讀前處理過後的資料
train = pd.read_csv('Processed_tbrain_training.csv')
print(train.shape)
print(train.info())
train.head(5)

In [ ]:
# 先取出每個顧客不重複最後一個月的值之後預測用
pre_train = train.drop_duplicates(subset=['chid'], keep='last')
pre_train

In [ ]:
# 去除shop_tag
pre_train.drop(columns = ['shop_tag'], axis=1, inplace=True)

In [ ]:
len(pre_train)

In [ ]:
predict = pd.DataFrame(pd.read_csv('需預測的顧客名單及提交檔案範例.csv'))
predict = predict.drop(columns = ['top1','top2','top3'])
print(predict.shape)
print(predict.info())
predict.head(5)

In [ ]:
# 按照要提交的檔案排序
pre_train = pre_train.set_index('chid', drop= False)
pre_train = pre_train.loc[predict.chid]
pre_train.head(5)

In [ ]:
# 拆分測試訓練集
from sklearn.model_selection import train_test_split
train, test = train_test_split(train, test_size=0.2, random_state=42)

In [ ]:
target_column_train = ['shop_tag']
predictors_train = list(set(list(train.columns))-set(target_column_train))

X_train = train[predictors_train].values
y_train = train[target_column_train].values

print(X_train.shape)
print(y_train.shape)

In [ ]:
target_column_test = ['shop_tag'] 
predictors_test = list(set(list(test.columns))-set(target_column_test))

X_test = test[predictors_test].values
y_test = test[target_column_test].values

print(X_test.shape)
print(y_test.shape)

In [ ]:
import gc
del train
del test
gc.collect()

In [ ]:
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1],1)))
model.add(LSTM(units=50))
model.add(Dense(49, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
import tensorflow as tf 
model.fit(X_train[0:1000000], y_train[0:1000000], epochs=2, batch_size=1, verbose=2)

In [ ]:
# 模型預測
predictions = model.predict(pre_train)

In [ ]:
len(predictions)

In [ ]:
len(predictions[0])

In [ ]:
# 按降序獲取前 n 個結果
best_n = np.argsort(-predictions, axis=1)[:, :49]
best_n

In [ ]:
# 建立best_predictions_final來儲存前三高的預測值
best_predictions_final = np.zeros((len(best_n), 3), dtype=int)

In [ ]:
# 過濾16個前三高的標籤出來
for a in range(len(best_n)):
    sum = 0
    for i in range(49): 
        if (best_n[a, i] == [2,6,10,12,13,15,18,19,21,22,25,26,39,37,39,48]).any():
            best_predictions_final[a, sum] = int(best_n[a, i])
            sum += 1 
        if sum == 3: 
            break

In [ ]:
submission = pd.DataFrame({ 'chid':predict.chid,'top1':best_predictions_final[0:len(best_predictions_final),0] , 'top2':best_predictions_final[0:len(best_predictions_final),1] , 'top3':best_predictions_final[0:len(best_predictions_final),2] })

In [ ]:
filename = 'submission.csv'
submission.to_csv(filename,index=False)
print(submission)
print('Saved file: ' + filename)

In [ ]:
# # 保存模型
# model.save('LSTM_model_200W-2.h5') 